In [1]:
"""
Author: Nils Weyrauch
Skript für die Erstellung des Bevölkerungsrasters auf Basis der Häuserklassifikation

Packages:
Geopandas
Pandas

Aufbau:
1. Ermittlung der prozentualen Anteile der Häuserklassen an den 100m Grid Zellen
2. Identifizierung geeigneter Kacheln mit 100% bedeckung einer Klasse für die Estimation von Bevölkerung/Voloumen für jede Häuserklasse
3. Mergen des des Datensatzes "Bevölkerung im 100m Gitter" auf die Kacheln in Polygon Form
4. Berechnung der Bevölkerung pro Volumen für die Häuserklassen
5. Berechnung der Bevölkerung für jedes Haus über die Bevölkerung pro Volumen Beziehung
6. Aufsummieren der Bevölkerung pro Gitterzelle
7. Prozentuale Abweichung von geschätzter zu tatsächlicher Bevölkerung pro Gitterzelle berechnen
"""

'\nAuthor: Nils Weyrauch\nSkript für die Erstellung des Bevölkerungsrasters auf Basis der Häuserklassifikation\n\nPackages:\nGeopandas\nPandas\n\nAufbau:\n1. Ermittlung der prozentualen Anteile der Häuserklassen an den 100m Grid Zellen\n2. Identifizierung geeigneter Kacheln mit 100% bedeckung einer Klasse für die Estimation von Bevölkerung/Voloumen für jede Häuserklasse\n3. Mergen des des Datensatzes "Bevölkerung im 100m Gitter" auf die Kacheln in Polygon Form\n4. Berechnung der Bevölkerung pro Volumen für die Häuserklassen\n5. Berechnung der Bevölkerung für jedes Haus über die Bevölkerung pro Volumen Beziehung\n6. Aufsummieren der Bevölkerung pro Gitterzelle\n7. Prozentuale Abweichung von geschätzter zu tatsächlicher Bevölkerung pro Gitterzelle berechnen\n'

In [2]:
import geopandas as gpd
import pandas as pd

In [3]:
"""
Funktion zur Ermittlung der prozentualen Häuserklassen Anteilen von den klassifizierten Häusern in Punktform in dem Polygonisierten 100m Grid für Deutschland

Aufbau:
1. Spatial Join der Punkte in jedes zugehörige Polygon
2. Gruppieren der Daten pro Polygon und pro Klasse
3. Berechnung der Prozentualen Anteile der jeweiligen Klasse an Gesamtzahl der Punkte in jedem Polygon
4. Transformation der Daten, sodass jeder Polygon ID die entsprechenden Klassen Anteile zugeordnet sind
5. Merge der Daten auf die ursprünglichen Polygon Daten sodass die Geometrien weiterhin vorhanden sind
"""

def calc_class_perc(poly_gdf, point_gdf, class_column):

    #sjoin der Punkt Daten auf die zugehörigen Polygone
    joined_data = poly_gdf.sjoin(point_gdf,how="left", predicate="intersects")
    
    # gruppieren der Daten nach Polygon index und Klassen, errechnen der Anzahl der Punkte pro Klasse
    grouped_data = joined_data.groupby(["id", class_column]).size()
    
    # gruppierte Daten werden umgewandelt, Index des Polygons werden Reihen und zugehörige Klassen-Zählung pro Polygon zu Splaten, NaN Values werden durch 0 ersetzt 
    grouped_piv = grouped_data.unstack(fill_value=0)
    
    #aufsummieren der Punkt Anzahl pro Polygon (Axis=1)
    point_sum = grouped_piv.sum(axis=1)

    # Berechnen der Prozentualen Anteile jeder Klasse für jedes Polygon
    piv_perc = grouped_piv.div(point_sum, axis=0) * 100

    # Jede Spalte erhält einen eindeutigen Namen, da die Werte in col in float angegeben werden müssen die Spalten Namen umgenannt werden für Ansicht im GIS
    piv_perc.columns = [f"Klasse_{col}_perc" for col in piv_perc.columns]
    piv_perc = piv_perc.rename(columns={"Klasse_1.0_perc" : "Klasse_1_perc", "Klasse_2.0_perc" : "Klasse_2_perc", "Klasse_3.0_perc" : "Klasse_3_perc", "Klasse_4.0_perc": "Klasse_4_perc", 
                                          "Klasse_5.0_perc" : "Klasse_5_perc", "Klasse_6.0_perc" : "Klasse_6_perc", "Klasse_7.0_perc" : "Klasse_7_perc" , "Klasse_8.0_perc" : "Klasse_8_perc"})

    # Erstellen der Spalte dom_class, welche angibt welche der class_perc Spalten den höchsten Prozentualen Wert enthält 
    # Die Prozent Spalten werden, mit iloc selektiert und in jeder Reihe wird der Spalten-Name des höchsten Wertes in der Spalte dom_class festgehalten
    piv_perc["dom_class"] = piv_perc.iloc[:, 0:8].apply(lambda x: x.idxmax(), axis=1)
    
    # idmax Funktion setzt den Namen der Spalte ein, die Spaltennamen werden per Mapping Dictionary durch die Klassen Werte ersetzt
    rename_dic = {"Klasse_1_perc" : "1", "Klasse_2_perc" : "2", "Klasse_3_perc" : "3", 
                  "Klasse_4_perc": "4", "Klasse_5_perc" : "5", "Klasse_6_perc" : "6", 
                  "Klasse_7_perc" : "7" , "Klasse_8_perc" : "8"}
    piv_perc["dom_class"] = piv_perc["dom_class"].map(rename_dic)
    
    # zurückmergen der Prozente an den ursprünglichen Datensatz um Geometrien wieder aufzunehmen
    merged_ip_gdf = poly_gdf.merge(piv_perc, how="left", left_on="id", right_on="id")

    return merged_ip_gdf
    

In [4]:
"""
Funktion zur Verknüpfung der Gridzellen mit den Kreisen des UG
Gridzellen werden Kreisen zugeordnet auf Basis der Gridzellen-Zentroide 

Aufbau:
1. Umwandeln der Gridzellen zu Zentroiden (Punkten), sodass eindeutige Zuordnung möglich ist
2. Spatial Join der UG Kreise auf die Gridzellen Zentroide
3. Mergen der Abkürzungen für die kreise des UGs auf IP Datensatz der Gridzellen
"""
def assign_ug_kreis(pol_grid, pol_lk):

    # Umwandeln der polygonisierten Grid-Zellen in Centroide, um diese eindeutig Kreisen zuordnen zu können 
    # vermeidet Probleme mit Zellen, die in mehreren Kreisen liegen
    grid_centr = pol_grid.copy()
    grid_centr["geometry"] = grid_centr.geometry.centroid
    
    # Verknüpfung der Grid Zellen mit dem entsprechenden Landkreis, über einen spatial join
    centr_sjoin = grid_centr.sjoin(pol_lk, how="left", predicate="intersects")
    
    # zurückmergen der Spalte mit den entsprechenden Landkreisen auf den ursprünglichen GDF
    grid_lk = pol_grid.merge(centr_sjoin["GN_ID"], how="left", left_index=True, right_index=True)

    return grid_lk

In [5]:
"""
funktion Berechnet Kovariante Bev/Vol für manuell gescoutete Zellen

Aufbau:
1. Filtern der Datensatzes, nach Gebieten die in trai gelabelt wurden
2. joinen der Häuser Daten auf die Grid Zelle
3. Summieren der Häuservolumen pro Zelle
4. Berechnung der Bev/Vol Beziehung der Zelle
5. Datensatz in Pivot Tabelle umbauen, welche die durchschnittliche Bev/Vol Beziehung pro Klasse und LK im UG aufspaltet
"""

def calc_Bev_vol_trained(grid_gdf, point_gdf):

    # Filter nach gelabelten Zellen
    grid_filt = grid_gdf[grid_gdf["Train"] == 1]

    # Joinen der Häuserdaten in Punktform auf den Griddatensatz
    grid_sjoin = grid_filt.sjoin(point_gdf, how="left", predicate="intersects")

    # Berechnung der Gebäudevolumen pro Zelle, Ergebnisspalte bekommt Namen "vol_sum"
    grid_vol_sum = grid_sjoin.groupby("id")["vol_adr"].sum().reset_index(name="vol_sum")

    # Die resultierende Series wird zurückgemergt
    grid_sjoin = grid_sjoin.merge(grid_vol_sum, on="id", how="left")

    # Berechnung der Einwohner pro summierten Volumen der Grid Zellen
    grid_sjoin["bev_vol"] = grid_sjoin["Einwohner"] / grid_sjoin["vol_sum"]

    # Ergebnisse werden in eine Pivot Tabelle überführt die als Index die LK Namen besitzen und als Spalten die Bev/Vol pro Hausklasse
    pivot = grid_sjoin.pivot_table(index="GN_ID", columns="dom_class", values="bev_vol", aggfunc="mean")

    return pivot


In [6]:
"""
Funktion zur Berechnung der Bev/Vol Beziehung über alle homogenen Häuser Grid Zellen, um unterschiede zu den ausgewählten Grid Zellen festzustellen

Aufbau:
1. Filtern der Datensatzes, nach Gebieten die 100 Prozent eine Häuserklasse aufweisen
2. joinen der Häuser Daten auf die Grid Zelle
3. Summieren der Häuservolumen pro Zelle
4. Berechnung der Bev/Vol Beziehung der Zelle
5. Datensatz in Pivot Tabelle umbauen, welche die durchschnittliche Bev/Vol Beziehung pro Klasse und LK im UG aufspaltet 
"""

def calc_Bev_vol_mean(grid_gdf, point_gdf):

    # Filtern des Datasets ob in den Werten (.any) der Spalten 8-15 die Prozentuale Anzahl 100 ist 
    # und ob die Einwohnerzahl 2011 in der Zelle verschlüsselt (-1) wurde, Zellen mit nur einem Haus werden ebenfalls rausgeschmissen um Verzerrungen zu vermeiden
    grid_filt = grid_gdf[((grid_gdf.iloc[:, 8:16] == 100.0).any(axis=1)) & (grid_gdf["Einwohner"] != -1) & (grid_gdf["House_Anz"] > 1)]
    
    # Joinen der Häuserdaten in Punktform auf den Griddatensatz
    grid_sjoin = grid_filt.sjoin(point_gdf, how="left", predicate="intersects")
    
    # Berechnung der Gebäudevolumen pro Zelle, Ergebnisspalte bekommt Namen "vol_sum"
    grid_vol_sum = grid_sjoin.groupby("id")["vol_adr"].sum().reset_index(name="vol_sum")
    
    # Die resultierende Series wird zurückgemergt
    grid_sjoin = grid_sjoin.merge(grid_vol_sum, on="id", how="left")
    
    # Berechnung der Einwohner pro summierten Volumen der Grid Zellen
    grid_sjoin["bev_vol"] = grid_sjoin["Einwohner"] / grid_sjoin["vol_sum"]
    
    # Ergebnisse werden in eine Pivot Tabelle überführt die als Index die LK Namen besitzen und als Spalten die Bev/Vol pro Hausklasse
    pivot = grid_sjoin.pivot_table(index="GN_ID", columns="dom_class", values="bev_vol", aggfunc="mean")

    return pivot

In [7]:
"""
Funktion zur Zuweisung der Werte der pivot Tabelle mit den Bev/Vol Werten pro Landkreis des Untersuchungsgebiets zu den entsprechenden Häuserklassen
Idee ist die Werte aus dem Häuser DF für die lokaliserung der Daten in der Pivot Tabelle zu nutzen. 
Die Werte entsprechenden den Feldern, in denen sich die richtigen Reihenwerte mit den richtigen Spaltenwerten überschneiden

Aufbau:
1. erste Schleife über Dataframe mit Häuserdaten, festlegen der zwei Spalten mit den Werten die den Reihen und Spalten der Pivot Tabelle entsprechen
2. if clause festlegen, wenn Match in index und Spalte der Pivot Tabelle gefunden, dann setze Werte in Spalte des Häuserdataframes
3. else clause festelgen, falls nicht gefunden
"""

def locate_val_piv(house_df, piv_bev_vol, op_column_name):

    #Erstellen der bev_vol Spalte im house df
    house_df[op_column_name] = 0.0

    # Schleife über house_df
    for index, row in house_df.iterrows():

        #Reihen (Kreis ID) und Spalten Werte (Häuserklasse des RF) festlegen für piv_bev_vol df
        row_val = row["GN_ID"]
        col_val = row["pred_int"]

        # If clause für die Erfüllung der zwei Bedingungen festlegen
        if (row_val in piv_bev_vol.index) and (col_val in piv_bev_vol.columns):

            # falls Bedingung erfüllt, weise entsprechenden Schnittpunkt Wert dem house_df zu
            match_val = piv_bev_vol.loc[row_val, col_val]
            house_df.loc[index, op_column_name] = match_val

        # else clause festlegen um Fehler zu identifizieren
        else:
            house_df.loc[index, op_column_name] = float("NaN")


    return house_df

In [8]:
"""
Funktion zur aufsummierung der geschätzten Hausbewohner pro Gridzelle

Aufbau:
1. Spatial Join der Häuser zu den jeweiligen Gridzellen
2. Datensatz gruppieren pro Grid ID und aufsummieren des angegeben Felds für die Bevölkerungszahlen
3. resultierende Series zurückmergen auf den GDF des grids per id
4. prozentuale Abweichung von offiziellen Daten berechnen und in neuer Spalte festhalten
"""

def sum_bev_in_grid(grid_gdf, haus_gdf, ip_bev_column,op_bev_sum_column, op_perc_dif_column):

    # Zuordnung der Häuser zu der jeweiligen Gridzelle
    grid_sjoin = grid_gdf.sjoin(haus_gdf, how="left", predicate="intersects")

    # Gruppieren des Datensatzes pro grid id, aufsummieren der Bevölkerungszahlen
    grid_bev_sum = grid_sjoin.groupby("id")[ip_bev_column].sum().reset_index(name=op_bev_sum_column)

    # mergen der Bevölkerungszahlen auf IP grid_gdf
    grid_gdf_merged = grid_gdf.merge(grid_bev_sum, on="id", how="left")

    # Berechnung der prozentualen Abweichung von offiziellen zu geschätzen Werten
    grid_gdf_merged[op_perc_dif_column] = (grid_gdf_merged["Einwohner"] - grid_gdf_merged[op_bev_sum_column]) /grid_gdf_merged["Einwohner"] * 100

    return grid_gdf_merged
    

In [9]:
"""
Funktion, welche den Gebäudeadressen das entsprechende Haus mit seinen Attributen zugewiesen wird. (analoges vorgehen zu Zensus Methodik)
Für Häuser mit mehreren Adressen wird das Volumen durch die Anzahl der Häuser geteilt, um Berechnungen von Häusern mit mehreren Adressen in verschiedenen Gridzellen nicht zu verzerren

Aufbau:
1. Filtern der wichtigen Spalten
2. Spatial Join der Hausattribute auf die Adresspunkte
3. Gruppieren des Datensatzes pro Haus ID
4. Zuordnung von neuer Volumen Spalte, welche aus Volumen/Anzahl der Häuser besteht
"""

def assign_house_attr(adress_gdf, house_gdf):

    # Filtern des Datensatzes auf eindeutige Kennung und entsprechende Geometrien, field_2 hält die oid der Adressen
    adress_filt = adress_gdf[["field_2", "geometry"]]

    # Joinen der Häuserattribute, inner join weil Häuser aus Datensatz entfernt wurden
    adress_sjoin = adress_filt.sjoin(house_gdf, how="inner", predicate="within")
    adress_sjoin.reset_index().drop_duplicates("index", inplace=True)
    
    #umbennen der Spalte field_2 zu oid, droppen der Spalte index_right um Fehler bei weiteren Joins zu vermeiden
    adress_sjoin.rename(columns={"field_2" : "oid"}, inplace=True)
    adress_sjoin.drop(columns=["index_right"], inplace=True)
    
    # Volumen der Häuser wird für jede Adresse durch Anzahl der Hausnummern geteilt und in neuer Spalte festgehalten
    adress_sjoin["vol_adr"] = adress_sjoin["VOL"] / adress_sjoin["HK_ANZ"]

    return adress_sjoin
    

In [10]:
def count_points_in_polygons(points, gdf, polygon_id="id"):

    # Zählen der Punkte in einem Polygon mittels räumlicher Verknüpfung, 
    # "inner" sorgt dafür, dass nur gezählt wird, falls Haus vorliegt, 
    # "groupby" gibt Attribut vor, für welches gezählt wird, hier: Häuseridentifier
    # "size" gibt die Anzahl der Punkte pro gml_id an, neue Spalte wird umbenannt
    point_per_pol = (gdf.sjoin(points, how="inner", predicate="intersects").groupby(polygon_id).size().rename("House_Anz"))

    # Merge der Häuser Anzahl auf die Gridzellen, für deren identifier "id"
    # NoData/NAN werden mit 0 gefüllt
    result_gdf = (gdf.merge(point_per_pol,on=polygon_id, how="left").fillna({"House_Anz": 0}))

    return result_gdf

In [11]:
"""
Funktion, welche den Identifier des Grids von 2022 in die Form des identifiers von dem Griddatensatz bringt

Aufbau:
slicen des Datensatzes sodass, Präfix CRS3035RES wegfällt, und 2 Stellen bei der Koordinatenbenennung
"""

def transform_identifier(identifier):
   
    # CRS3035RES fallenlassen
    identifier = identifier[10:]
    
    # Position von N und E Koordinaten feststellen für weiteren slice
    n_pos = identifier.find('N')
    e_pos = identifier.find('E')
    
    # Listen so slicen, dass die Parts die behaltem werden sollen, nur noch addiert werden müssen, immer die 5 Zahlen nach den Buchstaben
    part1 = identifier[:n_pos + 6]  
    part2 = identifier[e_pos:e_pos + 6]
    
    # identifier zusammenbringen
    new_identifier = part1 + part2
    
    return new_identifier
    

In [12]:
"""
Potenzieller Workflow des umwandeln der Gridzellen in UTM 32N und zuschnitt auf UG
wurde nicht verwendet aufgrund zu hoher Anforderungen an den Computer 

grid = gpd.read_file(r"D:\100m Grid DE\DE_Grid_ETRS89-LAEA_100m\geogitter\DE_Grid_ETRS89-LAEA_100m.gpkg")

# umwandeln des Koordinatensystems von ganzeuropäisch genormten, auf das passende für das UG
grid_reproj = grid.to_crs(epsg=25832)

# Auswahl der Gridzellen, welche teilweise in UG liegen
grid_ug = grid_reproj[grid_reproj.geometry.intersects(UG_kreise.unary_union)]
"""

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Nils\AppData\Local\Temp\ipykernel_9684\4029157439.py:1: SyntaxWarning: invalid escape sequence '\D'
  """


'\nPotenzieller Workflow des umwandeln der Gridzellen in UTM 32N und zuschnitt auf UG\nwurde nicht verwendet aufgrund zu hoher Anforderungen an den Computer \n\ngrid = gpd.read_file(r"D:@m Grid DE\\DE_Grid_ETRS89-LAEA_100m\\geogitter\\DE_Grid_ETRS89-LAEA_100m.gpkg")\n\n# umwandeln des Koordinatensystems von ganzeuropäisch genormten, auf das passende für das UG\ngrid_reproj = grid.to_crs(epsg=25832)\n\n# Auswahl der Gridzellen, welche teilweise in UG liegen\ngrid_ug = grid_reproj[grid_reproj.geometry.intersects(UG_kreise.unary_union)]\n'

In [13]:
# Import der benötigten Daten, grid = 100m Grid, RF_Haus = Ergebnisse der Random Forest Klassifikation
# geb_ref = georeferenzierte Adressen des UG

grid = gpd.read_file(r"F:\Basisdaten\grid_laea_ug_reproj.shp", crs="EPSG:25832")
RF_Haus = gpd.read_file(r"F:\Zwischendaten\RF_Results.shp", crs="EPSG:25832")
geb_ref = gpd.read_file(r"F:\Basisdaten\Geb_ref_UG.shp", crs="EPSG:25832")

#Herausfiltern von Häusern im Gebäudedatensat, welche zum Zeitpunkt der Höhenmodellaufnahme noch nicht da waren und dementsprechend kein Volumne besitzen (ca. 76)
Haus_filt = RF_Haus[RF_Haus["VOL"] != 0]

ip_kreise = gpd.read_file(r"F:\Basisdaten\dvg1krs_nw.shp", crs="EPSG:25832")
UG_kreise = ip_kreise[ip_kreise['GN'].isin([ "Ennepe-Ruhr-Kreis",  "Oberbergischer Kreis", "Wuppertal", "Remscheid" ])]

In [22]:
# Import des ursprünglichen Hausdatensatzes, welcher später eine zuordnung der Gebäudefunktion erlaubt
ALKIS_house = gpd.read_file(r"F:\Zwischendaten\Data_Hausmerkmale.shp")

In [15]:
# aus Datensatz gefilterte Hochhäuser wieder einbringen
hochhaus = gpd.read_file(r"F:\Zwischendaten\hochhaus.shp", crs="EPSG:25832")

In [16]:
#Da Hochhäuser nicht häufig genug vorhanden sind um eigene Werte zu entwickeln, werden diese zu Klasse 7 (Plattenbauten) reklassifiziert, 
#da deren strukturelle Merkmale denen der Hochhäuser am meisten ähneln
hochhaus["PREDICTED"] = 7.0
hochhaus["VOL"] = hochhaus["vol"]

In [17]:
# um Spalten zu vermeiden die überflüssig sind, check auf die Überschneidung und nur verschnitt dieser
inters_col = Haus_filt.columns.intersection(hochhaus.columns)
haus_inters = Haus_filt[inters_col]
hochhaus_inters = hochhaus[inters_col]

In [18]:
#Hochhäuser werden an Klassifizierten Datensatz angehängt
Haus = pd.concat([haus_inters, hochhaus_inters])

In [19]:
#Import des Datensatzes Bevölkerung im 100m Gitter (.csv) und umwandlung in einen Dataframe
bev_grid = pd.read_csv(r"F:\Basisdaten\Zensus2022_Bevoelkerungszahl_100m-Gitter.csv", sep=";")
bev_grid_df = pd.DataFrame(bev_grid)

In [20]:
# neuen Identifier dem alten anpassen
bev_grid_df['GITTER_ID_100m'] = bev_grid_df['GITTER_ID_100m'].apply(transform_identifier)

In [23]:
# Herausfiltern von Duplikaten, Normalize stellt sicher, dass das Filterung richtig funktioniert, indem es die Polygon Vertices Reihenfolgen einheitlich ordnet,
ALKIS_house.normalize()
ALKIS_clean = ALKIS_house.drop_duplicates(subset='geometry')

In [24]:
Haus.normalize()
haus_clean = Haus.drop_duplicates("geometry")

In [25]:
# Filtern des Datensatzes auf die relevanten Spalten, und zur vermeidung von Dopplung bei Sjoin
ALKIS_house_filt = ALKIS_clean[["gml_id", "gebaeudefu", "geometry"]]

In [26]:
# sjoin der Haus Punkte, um die Gebäudefunktion und gml_id der Gebäude, welche während der RF Klassifikation verloren gehen,
# wieder den Häusern zuzuweisen, erlaubt später Korrektur Faktor zu nutzen für Häuser die neben einer Wohnnutzung auch eine 
# wirtschaftliche Nutzung aufweisen
Haus_gebfun = gpd.sjoin(haus_clean, ALKIS_house_filt, how="left", predicate="intersects")

#Sjoin ordnet nicht eins zu eins zu, Duplikate die durch Join entstanden sind werden gefiltert
Haus_gebf_dupl = Haus_gebfun.reset_index().drop_duplicates("index")

In [27]:
# vermeidet Fehler bei zukünfitgen Joins, wenn das Feld Index_right gedroppt wird
Haus_gebf_dupl.drop(columns=["index"], inplace=True)
Haus_gebf_dupl.drop(columns=["index_right"], inplace=True)

In [28]:
# Anlegen einer Abkürzungsspalte für die Namen der Landkreise
# erlaubt später Zuordnung der Grid Zellen zu einem Gemeinde Gebiet per Funktion
LK_ID = {"Remscheid" : "RS", "Wuppertal" : "WU", "Oberbergischer Kreis" : "OBK", "Ennepe-Ruhr-Kreis" : "ERK"}
UG_kreise["GN_ID"] = UG_kreise["GN"].map(LK_ID)

C:\Users\Nils\anaconda3\envs\geopandas\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
# Zuweisen der Häuserattribute auf die entsprechenden Adresspunkte, 
# nötig um die selben Punkte zur Hochrechnung der Bevölkerung zu nutzen wie statistisches Bundesamt
adr_point = assign_house_attr(geb_ref, Haus_gebf_dupl)

In [30]:
# den adressen wird ihrer lage entsprechend der richtige Landkreis zugeordnet
adr_point_GN = assign_ug_kreis(adr_point, UG_kreise)

In [31]:
# Kalkulation der prozentualen Anteile der RF klassifizierten Häuser an jeder Gridzelle um homogene Gebiete zu identifizieren
perc = calc_class_perc(grid, adr_point, "PREDICTED")

In [32]:
# Fallen lassen von Spalten, die nicht benötigt werden
drop_col = ["featuretyp", "dataset_na", "f_staat", "f_land", "f_wasser", "p_staat", "p_land", "p_wasser", "Shape_Leng", "Shape_Area","OBJECTID"]
perc_cl = perc.drop(columns=drop_col)

In [33]:
# Mergen der Datensätze mit prozentualen Anteilen der Häuserklassen und der Bevölkerung pro Gridzelle
grid_perc_bev = perc_cl.merge(bev_grid_df, how="inner", left_on="id", right_on="GITTER_ID_100m")
# Aussortieren der Kacheln, die aufgrund von Datenschutz verschlüsselt sind oder nicht bewohnt sind
grid_perc_bev = grid_perc_bev[grid_perc_bev["Einwohner"] != -1]

In [34]:
# Zuordnung der Gridzelle, zu dem jeweiligen Landkreis, in dem sich die Gridzellen Mitte befindet
grid_perc_GN = assign_ug_kreis(grid_perc_bev, UG_kreise)

In [35]:
# zählen der Häuser in jeder Gridzelle 
grid_perc_count = count_points_in_polygons(adr_point, grid_perc_GN)

In [36]:
grid_perc_count.to_file(r"F:\Zwischendaten\grid_ug_perc_2022.shp")

C:\Users\Nils\AppData\Local\Temp\ipykernel_9684\162236167.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_perc_count.to_file(r"F:\Zwischendaten\grid_ug_perc_2022.shp")


In [37]:
# Berechnung der Bevölkerung / Volumen Beziehung für jede Gridzelle die 100 Prozent eine Häuserklasse aufweist
# erlaubt spätere Gegenüberstellung mit manuell gescouteten Gebieten
bev_vol_mean = calc_Bev_vol_mean(grid_perc_count, adr_point)

In [38]:
bev_vol_mean.head()

dom_class,1,2,3,5,6,7,8
GN_ID,,,,,,,
ERK,0.000982,0.001097,0.001265,0.001233,0.001260,0.001301,0.001217
OBK,0.000957,0.000962,0.001160,0.001023,0.001086,0.001173,0.000754
RS,0.001016,0.001026,0.001340,0.001199,0.001324,0.001465,0.001135
WU,0.000981,0.001069,0.001308,0.001092,0.001495,0.001503,0.001208


In [39]:
# Export der Pivot Tabelle, die die durchschnittliche Bev/Vol Beziehung angibt
bev_vol_mean.to_csv(r"F:\Zwischendaten\pivot_bev_vol_mean.csv")

In [40]:
# Zuweisen der Landkreis IDs auf die Häuser über das Grid (sodass Zuordnung von Haus und Grid zu selbem LK gewährleistet ist) 
# für Zuordnung von regional spezifischen Werten, der index den die Funktion Sjoin automatisch erstellt, wird gedropt
haus_gn = adr_point.sjoin(grid_perc_GN[["geometry","GN_ID"]], how="left", predicate="intersects").drop(columns="index_right")

In [41]:
haus_gn["pred_int"] = haus_gn["PREDICTED"].astype(int)

In [42]:
# umwandeln der Spalten Namen zu Integern der bev_vol_mean Spaltennamen, da sonst zuordnung mit den Integer Werten der Daten nicht funktioniert
bev_vol_mean.columns = bev_vol_mean.columns.astype(int)

In [43]:
# Zuordnen des regionalen Bev/Vol Wert für die Ergebnisse der RF Klassifikation
haus_bev = locate_val_piv(haus_gn, bev_vol_mean, "bev_vol_mean")

In [44]:
# Hochrechnung der Anzahl der Hausbewohner auf Basis des Volumens pro Adresse und der Bev/Vol Beziehung der homogenen Kacheln
haus_bev["Bev_Haus_m"] = haus_bev["vol_adr"] * haus_bev["bev_vol_mean"]

In [45]:
# Einbezug eines Korrektur Faktors für Gebäude, welche nicht nur Wohnnutzung aufweisen über das Attribut "gebaeudefun"
# nummern in Liste stehen für alle Attribut Werte, welche mit Wohn und wirtschaftliche Nutzung ausgewiesen sind ca. 18 000

for index, row in haus_bev.iterrows():
    if row["gebaeudefu"] in [1110, 1120, 1121, 1122, 1123, 1130, 1131, 2310, 2320]:
        haus_bev.at[index, "Bev_Haus_m"] *= 0.75

In [46]:
# Aufsummieren der Anzahl der geschätzten Hausbewohner pro Gridzelle und Berechnung der Abweichung
grid_bev_abw_m = sum_bev_in_grid(grid_perc_count, haus_bev, "Bev_Haus_m", "m_pred_bev_sum", "dif_bev_m")

In [47]:
#export des grids um Zellen zu identifizieren die für zweite Modell Variante geeignet sind
#grid_bev_abw_m.to_file(F:\Zwischendaten\grid_bev_abw_2022.shp")

In [49]:
# Berechnung der Bevölkerung/Volumen Beziehung auf Basis ausgesuchter Gridzellen ohne Fehlerhafte Klassifikationen zum Vergleich mit den über den durchschnitt ermittelten Bevölkerungs-Volumen Beziehung

# Import der gesichteten Gridzelle, welche mit dem Feld "Trained" gekennzeichnet wurden
grid_trained = gpd.read_file(r"F:\Zwischendaten\grid_ug_perc_trained.shp")

In [51]:
# Da die Sichtung der Zellen für das Trained Modell auf Basis des 2011 grids erfolgte, müssen die gescouteten Zellen auf den aktualisierten Datensatz gemergt werden
grid_trained_new = grid_bev_abw_m.merge(grid_trained[["Gitter_ID_","Train"]], left_on="GITTER_ID_100m", right_on="Gitter_ID_")

In [52]:
# Berechnung der Bevölkerung-Volumen Beziehung für die trainierten Gridzellen
bev_vol_trained = calc_Bev_vol_trained(grid_trained_new, adr_point)

In [53]:
# Export der Pivot Tabelle zum Vergleich mit den über den Durchschnitt ermittelten Daten
bev_vol_trained.to_csv(r"F:\Zwischendaten\pivot_bev_vol_trained_2022.csv")

In [54]:
# umwandeln der Spalten Namen zu Integern der bev_vol_mean Spaltennamen, da sonst zuordnung mit den Integer Werten der Daten nicht funktioniert
bev_vol_trained.columns = bev_vol_trained.columns.astype(int)

In [55]:
# Zuordnen des regionalen Bev/Vol Wert für die Ergebnisse der RF Klassifikation
haus_bev = locate_val_piv(haus_gn, bev_vol_trained, "bev_vol_trained")

In [56]:
# Hochrechnung der Anzahl der Hausbewohner auf Basis des Volumens pro Adresse und der Bev/Vol Beziehung der trainierten Kacheln
haus_bev["Bev_Haus_t"] = haus_bev["vol_adr"] * haus_bev["bev_vol_trained"]

In [57]:
# Einbezug eines Korrektur Faktors für Gebäude, welche nicht nur Wohnnutzung aufweisen über das Attribut "gebaeudefun"
# nummern in Liste stehen für alle Attribut Werte, welche mit Wohn und wirtschaftliche Nutzung ausgewiesen sind ca. 18 000

for index, row in haus_bev.iterrows():
    if row["gebaeudefu"] in [1110, 1120, 1121, 1122, 1123, 1130, 1131, 2310, 2320]:
        haus_bev.at[index, "Bev_Haus_t"] *= 0.75

In [58]:
# Aufsummieren der Anzahl der geschätzten Hausbewohner pro Gridzelle und Berechnung der Abweichung für die trainierten Kacheln
grid_bev_abw_t = sum_bev_in_grid(grid_perc_count, haus_bev, "Bev_Haus_t", "t_pred_bev_sum", "dif_bev_t")

In [59]:
# mergen der beiden Datensätze mit den verschiedenen Bevölkerungsschätzungen, um diese im folgenden zusammenzufassen und auszuwerten
grid_bev_abw = grid_bev_abw_m.merge(grid_bev_abw_t[["id", "t_pred_bev_sum", "dif_bev_t"]], on="id", how="left")

In [60]:
# Export der Daten
grid_bev_abw.to_file(r"F:\Zwischendaten\grid_bev_abw_merge_2022.shp")

C:\Users\Nils\AppData\Local\Temp\ipykernel_9684\3356142617.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_bev_abw.to_file(r"F:\Zwischendaten\grid_bev_abw_merge_2022.shp")


In [61]:
adr_point_GN.to_file(r"F:\Zwischendaten\adr_point.shp")